In [117]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
import numpy as np
import cv2
import mss
import time
import os
import warnings
import logging
from concurrent.futures import ThreadPoolExecutor
import pathlib

In [118]:
# CONSTANTS

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
VID_OUTPUT_DIR = 'video_out'        # Directory to save video output
DETECT = True                       # Whether or not to label during capture process, False to simply record and output videos. No output if true
FPS = 15                            # Frames to capture per second
SECONDS_TO_CAPTURE = 60             # Video duration

In [119]:
if DEVICE == torch.device('cuda'):
    print(f'Using {DEVICE} {torch.cuda.get_device_name()} with {torch.cuda.mem_get_info()[0]/(1024**2)} MB of VRAM')

Using cuda NVIDIA GeForce RTX 3050 Ti Laptop GPU with 3110.4500007629395 MB of VRAM


In [120]:
if not os.path.exists('yolov5'):
  !git clone https://github.com/ultralytics/yolov5
  !pip install -r yolov5/requirements.txt
  

if not os.path.exists(VID_OUTPUT_DIR):
  os.makedirs(VID_OUTPUT_DIR)
  
warnings.simplefilter("ignore", FutureWarning)
logging.getLogger('ultralytics').setLevel(logging.ERROR)

In [121]:
pathlib.PosixPath = pathlib.WindowsPath # https://github.com/ultralytics/yolov5/issues/10240#issuecomment-1662573188
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./models/best.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\tiany/.cache\torch\hub\master.zip
YOLOv5  2024-10-30 Python-3.10.6 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [ ]:
def detect(img, model):
  lanes = {
    0 : (10, 180),
    1 : (150, 320),
    2 : (300, 470),
    3 : (440, 610)
  }
  ret = []  
  res = model(img)
  
  for box in res.xyxy[0]:
    # Confidence level is less than 50%
    if box[4] < 0.50:
      continue
    
    x_center = int((box[0] + box[2]) / 2)
    y_center = int((box[1] + box[3]) / 2)
    class_id = int(box[5]) # classes are 0: end_hold, 1: note, 2: start_hold
    
    # Identify the lane of the note based on x_center
    for lane, (start, end) in lanes.items():
      if start <= x_center <= end:
        break
    
    ret.append([class_id, lane, y_center])
  
  return ret 

def capture(region):
  with mss.mss() as sct:
    return sct.grab(region)

In [ ]:
frames_to_capture = FPS * SECONDS_TO_CAPTURE
frames = []

monitor_1 = mss.mss().monitors[1]
print(f'Monitor: {monitor_1}')
t, l, w, h = monitor_1['top'], monitor_1['left'], monitor_1['width'], monitor_1['height']
region_1 = {'left': l+int(w * 0.338), 'top': t, 'width': w-int(w * 0.673), 'height': h} # Gameplay region
# region_2 = {'left': l+int(w * 0.67), 'top': t, 'width': w-int(w * 0.98), 'height': h-int(h * 0.33)} # Judgement counter region
# region_3 = {'left': l+int(l * 0.859), 'top': t, 'width': w-int(w * 0.820), 'height': h-int(h * 0.952)} # Score region

print(f'Capturing {region_1}')

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_1 = cv2.VideoWriter(f'{VID_OUTPUT_DIR}/output_1.mp4', fourcc, FPS, (region_1['width'], region_1['height']))

with ThreadPoolExecutor(max_workers=8) as executor:
  for i in range(frames_to_capture):
    time_start = time.time()
    thread_1 = executor.submit(capture, region_1)
    img_1 = thread_1.result()
    
    if DETECT:
      thread_1 = executor.submit(detect, np.array(img_1), model)
      notes = thread_1.result() # Outputs notes in [[class_id, lane_num, y_center_pos]...]
      print(notes)
    else:
      out_1.write(cv2.cvtColor(np.array(img_1), cv2.COLOR_BGRA2BGR))
    
    # If the current loop finishes faster than the FPS, wait.
    if time_start + 1/FPS > time.time():
      time.sleep(time_start + 1/FPS - time.time())

if not DETECT:
  out_1.release()
  print(f'Capture region saved to {VID_OUTPUT_DIR}/output_1.mp4')

Monitor: {'left': 1920, 'top': 0, 'width': 1920, 'height': 1080}
Capturing {'left': 2568, 'top': 0, 'width': 628, 'height': 1080}
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[[1, 24], [0, 23]]
[[3, 33], [1, 140], [0, 140]]
[[1, 307], [0, 307], [2, 45], [3, 162]]
[[2, 154], [1, 442], [3, 299], [0, 41], [0, 442]]
[[2, 322], [3, 467], [1, 612], [0, 612], [1, 53], [0, 180]]
[[2, 492], [1, 205], [3, 638], [2, 65], [3, 64], [1, 781], [0, 781], [0, 349]]
[[2, 628], [2, 194], [1, 341], [3, 774], [1, 916], [3, 194], [0, 916], [0, 486]]
[[2, 799], [2, 365], [3, 366], [0, 655], [1, 512]]
[[1, 682], [0, 823], [2, 538], [3, 535]]
[[1, 816], [2, 673], [3, 672], [0, 957], [3, 12], [1, 12]]
[[1, 120], [3, 119], [3, 843], [2, 832]]
[[2, 77], [1, 257], [3, 257]]
[[1, 426], [2, 247], [0, 69], [3, 426]]
[[1, 596], [1, 65], [2, 417], [3, 596], [0, 237]]
[[3, 732], [2, 553], [1, 733], [1, 195], [0, 374], [2, 44]]
[[2, 726], [3, 903], [1, 365], [0, 544], [2, 174]]
[[1, 536], [2, 894], [0, 716

In [130]:
res = model('frames\\unlabeled\\1729608856\\frame_00016.jpg')

In [131]:
res.xyxy

[tensor([[453.57294,  24.92634, 598.30945, 171.98288,   0.93061,   1.00000],
         [ 26.81798, 429.72552, 169.10820, 574.27899,   0.92501,   1.00000]], device='cuda:0')]